In [1]:

import datetime
import datetime as dt
from netCDF4 import date2num,num2date
from math import sqrt


import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc
from sklearn import preprocessing
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# rdata_daily = xr.open_dataset('/content/drive/MyDrive/ERA5_meteo_sfc_2021_daily.nc') #.rename({'latitude':'lat','longitude':'lon'})
# #rdata_daily = rioxarray.open_rasterio('/content/drive/MyDrive/ERA5_meteo_sfc_2021_daily.nc').rename({'y':'lon','x':'lat'})
# rdata_daily

In [4]:
path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_daily.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_daily_smalldomain.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_hourly.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_hourly_smalldomain.nc')
data = xr.open_dataset(path2, decode_times=False) #To view the date as integers of 0, 1, 2,....
#data = xr.open_dataset(path2)# decode_times=False) #To view the date as integers of 0, 1, 2,....
#data5 = xr.open_dataset(path2) # To view time in datetime format
data

<xarray.Dataset>
Dimensions:    (time: 365, longitude: 41, latitude: 41)
Coordinates:
  * time       (time) float64 0.0 1.0 2.0 3.0 4.0 ... 361.0 362.0 363.0 364.0
  * longitude  (longitude) float32 -33.0 -32.75 -32.5 ... -23.5 -23.25 -23.0
  * latitude   (latitude) float32 43.0 42.75 42.5 42.25 ... 33.5 33.25 33.0
Data variables:
    sp         (time, latitude, longitude) float64 ...
    sst        (time, latitude, longitude) float64 ...
    u10        (time, latitude, longitude) float64 ...
    v10        (time, latitude, longitude) float64 ...
    sshf       (time, latitude, longitude) float64 ...
    slhf       (time, latitude, longitude) float64 ...
    t2m        (time, latitude, longitude) float64 ...
Attributes:
    title:          Daily variables from ERA5 hourly data
    history:        Wed Mar 23 12:01:11 2022: ncrcat ERA5_meteo_sfc_2021_01-0...
    Input dataset:  /global/cscratch1/sd/xzheng/ENA/rawdata/ERA5_meteo_sfc_20...
    NCO:            netCDF Operators version 5.0.1 (Homepage = http://nco.sf....

In [5]:
formed_clusters=np.array([4, 4, 4, 4, 2, 2, 2, 2, 2, 4, 4, 0, 4, 4, 0, 4, 0, 0, 0, 0, 0, 4,4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 2, 2, 4, 4, 0, 0, 0, 0, 0, 0,3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 2, 4, 4, 4, 0, 0, 0, 0, 4, 4, 4, 4,4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 2, 4, 4, 0,3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 0,0, 3, 3, 2, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 0, 0, 4, 2, 4, 4, 4,4, 4, 4, 4, 6, 4, 4, 6, 2, 2, 6, 6, 4, 6, 6, 6, 6, 6, 6, 2, 6, 6,6, 6, 6, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1,1, 1, 1, 1, 1, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5,1, 5, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5,5, 5, 1, 1, 5, 1, 1, 1, 1, 2, 5, 5, 5, 5, 1, 6, 6, 6, 6, 6, 2, 2,6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 3, 2, 6, 4, 0, 3, 3,3, 3, 0, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3])         

In [6]:
formed_clusters

array([4, 4, 4, 4, 2, 2, 2, 2, 2, 4, 4, 0, 4, 4, 0, 4, 0, 0, 0, 0, 0, 4,
       4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 2, 2, 4, 4, 0, 0, 0, 0, 0, 0,
       3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 2, 4, 4, 4, 0, 0, 0, 0, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 2, 4, 4, 0,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 0,
       0, 3, 3, 2, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 0, 0, 4, 2, 4, 4, 4,
       4, 4, 4, 4, 6, 4, 4, 6, 2, 2, 6, 6, 4, 6, 6, 6, 6, 6, 6, 2, 6, 6,
       6, 6, 6, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1,
       1, 1, 1, 1, 1, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1,

# Download the full repository from Github in ZIP format. Then upload that ZIP folder in the Colab memory.

In [7]:
!git clone https://github.com/big-data-lab-umbc/multivariate-weather-data-clustering.git

Cloning into 'multivariate-weather-data-clustering'...
fatal: could not read Username for 'https://github.com': No such device or address


In [8]:
!unzip /content/drive/MyDrive/Github_notebooks/Tests/multivariate-weather-data-clustering-main.zip

Archive:  /content/drive/MyDrive/Github_notebooks/Tests/multivariate-weather-data-clustering-main.zip
5b62d1afad6c4cc801357e9c47185da73a08f37e
   creating: multivariate-weather-data-clustering-main/
  inflating: multivariate-weather-data-clustering-main/.DS_Store  
  inflating: multivariate-weather-data-clustering-main/.gitignore  
   creating: multivariate-weather-data-clustering-main/Benchmark/
  inflating: multivariate-weather-data-clustering-main/Benchmark/.DS_Store  
   creating: multivariate-weather-data-clustering-main/Benchmark/Affinity_Propagation/
  inflating: multivariate-weather-data-clustering-main/Benchmark/Affinity_Propagation/Affinity_Propagation_PCA11_Norm.ipynb  
   creating: multivariate-weather-data-clustering-main/Benchmark/Dbscan/
  inflating: multivariate-weather-data-clustering-main/Benchmark/Dbscan/Git_testing_DBscan_SST_only.ipynb  
  inflating: multivariate-weather-data-clustering-main/Benchmark/Dbscan/Git_testing_DBscan_all_variables_.ipynb  
  inflating: mu

In [9]:
!mv multivariate-weather-data-clustering-main multivariateweatherdataclusteringmain

In [10]:
from multivariateweatherdataclusteringmain.MWDC.preprocessing import *
# #from multivariateweatherdataclusteringmain.MWDC.Evaluation.sp_evaluation_2 import *


#Spatio-Temporal RMSE

def ST_RMSE(input,formed_clusters,normalize=False):

  '''
  input: 
        datatype: 4-D spatio-temporal xarray

        formed_clusters: 1-D array of cluster labels classifying each data point along the time dimension
                         to a cluster label

  Output:
         
        An N X M matrix whose diagonal is a measure of intra-rmse between data points in a cluster
        while the rest of the values represent the inter-rmse between data points in different clusters.
     
  '''

  trans_data = datatransformation(input)

  trans_data = datanormalization(trans_data)

  trans_data['Cluster'] = formed_clusters
    
  # Function to get center of dataset (compute the mean value of all centroids)

  # Non-normalized
  # Function that creates a dictionary that holds all the cluster centers
  def data_centroid(input,formed_clusters): #classification
    Cluster_Centers = {}
    centers = []
    Center = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    for i in set(formed_clusters):
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      centers.append(Cluster_Centers['Cluster_Center' + str(i)])

    centers_sum = sum(centers)
    centers = []
    n = len(Cluster_Centers['Cluster_Center' + str(i)])
    Centers_sum = np.sum(sum(sum(centers_sum/n)))
    Center.append(Centers_sum)

    return Center


  # Non-normalized
  # Function that creates a dictionary that holds the values of dates in each cluster
  def get_datewise_clusters(formed_clusters): # classification
    Dates_Cluster = {}
    for i in set(formed_clusters): # classification
      Dates_Cluster['Dates_Cluster'+str(i)] = trans_data.index[trans_data.Cluster == i].to_list()
    return Dates_Cluster



  # Non-normalized
  # Function that creats a dictionary that holds all the clusters
  def n_nor_get_clusters(input,formed_clusters): # classification
    com_arr = []
    Clusters = {}
    Dates_Cluster = get_datewise_clusters(formed_clusters)
    for i in set(formed_clusters):
      for j in Dates_Cluster['Dates_Cluster'+str(i)]:
        arr = np.array(input.isel(time=j).to_array()) # input is data
        com_arr.append(arr)
      Clusters['Cluster' + str(i)] = np.array(com_arr)
      com_arr = []
    return Clusters


  # Non-normalized
  # Function that creates a dictionary that holds all the cluster centers
  def n_nor_get_cluster_centers(input,formed_clusters): #classification
    Cluster_Centers = {}
    Clusters = n_nor_get_clusters(input,formed_clusters)
    for i in set(formed_clusters):
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Cluster_Centers



  # Non-normalized 
  #Intra RMSE Calculation Function
  def n_nor_intra_rmse(input,formed_clusters):
    sq_diff = []
    intra_rmse = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)

    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sq_diff)
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(sum(sum(sum(Sq_diff_sum/n))))
      intra_rmse.append(Sqrt_diff_sum)
    return intra_rmse

  # Normalized
  # Function that creates two dictionaries that hold all the clusters and cluster centers
  def nor_get_clusters_and_centers(input,formed_clusters):
    Clusters = {}
    Cluster_Centers = {}
    for i in set(formed_clusters):
      Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Clusters,Cluster_Centers


  # Normalized

  def nor_intra_rmse(input,formed_clusters):
    intra_rmse = []
    sq_diff = []
    Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
    for i in range(len(Clusters)):
      for j in range(len(Clusters['Cluster' + str(i)])):
        diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
        Sq_diff = (diff**2)
        sq_diff.append(Sq_diff)
      Sq_diff_sum = sum(sum(sq_diff))
      sq_diff = []
      n = len(Clusters['Cluster' + str(i)])
      Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
      intra_rmse.append(Sqrt_diff_sum)
    return intra_rmse



  # RMSE Calculation
  def RMSE(input,formed_clusters,normalize=False):
    inter_rmse = []
    avg_cluster = {}

    if normalize == False:
      input = handle_missing_values(input)
      Clusters = n_nor_get_clusters(input,formed_clusters)
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_rmse(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
            Sq_diff = (diff**2)
            #Sq_diff_sum = sum(Sq_diff)
            Sq_diff_sum = sum(sum(sum(Sq_diff)))
            #inter_rmse.append(np.sqrt(Sq_diff_sum))
            n = len(avg_cluster['avg_cluster'+str(i)][0])
            Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
            mat[i].iloc[j] = round(Sqrt_diff_sum,2)
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
          

    else:
      # trans_data = datatransformation(input)

      # # Data Normalization
      # trans_data = datanormalization(trans_data)

      # # Adding class centers and cluster numbers as columns to the dataframe
      # trans_data['Cluster'] = formed_clusters

      # Rearranging the columns in the dataframe
      # trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
    
      Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

      # Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
      #trans_data = trans_data.reset_index()

    
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = nor_intra_rmse(trans_data,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
            Sq_diff = (diff**2)
            #Sq_diff_sum = sum(Sq_diff)
            Sq_diff_sum = sum(Sq_diff)
            #inter_rmse.append(np.sqrt(Sq_diff_sum))
            Sqrt_diff_sum = np.sqrt(Sq_diff_sum)
            mat[i].iloc[j] = round(Sqrt_diff_sum,2)
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())

    return mat
  return RMSE(input,formed_clusters,normalize=True)






#################################################################################################
#################################################################################################







#Spatio-Temporal Correlation


#Spatio-Temporal Correlation


def ST_CORRELATION(input,formed_clusters,normalize=True):

  '''
  input: 
        datatype: 4-D spatio-temporal xarray

        formed_clusters: 1-D array of cluster labels classifying each data point along the time dimension
                         to a cluster label

  Output:
         
        An N X M matrix whose diagonal is a measure of intra-spatial correlation between data points in a cluster
        while the rest of the values represent the inter-spatial correlation between data points in different clusters.
     
  '''

  trans_data = datatransformation(input)

  trans_data = datanormalization(trans_data)

  trans_data['Cluster'] = formed_clusters

  def pearson_PM(x, y):

    #convert format from netcdf to np array
    #x_form = x.to_numpy()
    #y_form = y.to_numpy()

    #Flatten/transform from 2d to 1d
    X_flat = x.flatten()
    Y_flat = y.flatten()

    #Compute correlation matrix
    corr_mat = np.corrcoef(X_flat, Y_flat)

    #Return entry [0,1]
    return corr_mat[0,1]
  

  # Function to get center of dataset (compute the mean value of all centroids)

  # Non-normalized
  # Function that creates a dictionary that holds all the cluster centers
  def data_centroid(input,formed_clusters): #classification
    Cluster_Centers = {}
    centers = []
    Center = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    for i in set(formed_clusters):
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      centers.append(Cluster_Centers['Cluster_Center' + str(i)])

    centers_sum = sum(centers)
    centers = []
    n = len(Cluster_Centers['Cluster_Center' + str(i)])
    Centers_sum = np.sum(sum(sum(centers_sum/n)))
    Center.append(Centers_sum)

    return Center


  # Non-normalized
  # Function that creates a dictionary that holds the values of dates in each cluster
  def get_datewise_clusters(formed_clusters): # classification
    Dates_Cluster = {}
    for i in set(formed_clusters): # classification
      Dates_Cluster['Dates_Cluster'+str(i)] = trans_data.index[trans_data.Cluster == i].to_list()
    return Dates_Cluster



  # Non-normalized
  # Function that creats a dictionary that holds all the clusters
  def n_nor_get_clusters(input,formed_clusters): # classification
    com_arr = []
    Clusters = {}
    Dates_Cluster = get_datewise_clusters(formed_clusters)
    for i in set(formed_clusters):
      for j in Dates_Cluster['Dates_Cluster'+str(i)]:
        arr = np.array(input.isel(time=j).to_array()) # input is data
        com_arr.append(arr)
      Clusters['Cluster' + str(i)] = np.array(com_arr)
      com_arr = []
    return Clusters


  # Non-normalized
  # Function that creates a dictionary that holds all the cluster centers
  def n_nor_get_cluster_centers(input,formed_clusters): #classification
    Cluster_Centers = {}
    Clusters = n_nor_get_clusters(input,formed_clusters)
    for i in set(formed_clusters):
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Cluster_Centers



  # Non-normalized 
  #Intra-spatial correlation coefficient Calculation Function
  import functools 

  def n_nor_intra_sp_corr(input,formed_clusters):
    mylist = []
    intra_sp_corr = []
    Clusters = n_nor_get_clusters(input,formed_clusters)
    Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
    
    for i in range(len(Clusters)):
      mylist = []
      for j in range(len(Clusters['Cluster' + str(i)])):
        corr_coeff = pearson_PM(Clusters['Cluster' + str(i)][j], Cluster_Centers['Cluster_Center' + str(i)])
        #print('i: {}, j: {}, corr_coeff:{}'.format(i, j, corr_coeff))
        mylist.append(corr_coeff)
        average_corr_coeff = sum(mylist) / len(mylist)
      intra_sp_corr.append(average_corr_coeff)
    return intra_sp_corr

  

    #Intra-spatial correlation coefficient Calculation Function
  import functools 

  def sp_corr(input,formed_clusters):
    avg_cluster = {}
    Clusters = n_nor_get_clusters(input,formed_clusters)
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = n_nor_intra_sp_corr(input,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
          mat[i].iloc[j] = corr_coeff
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
      return mat

  # Normalized
  # Function that creates two dictionaries that hold all the clusters and cluster centers
  def nor_get_clusters_and_centers(input,formed_clusters):
    Clusters = {}
    Cluster_Centers = {}
    for i in set(formed_clusters):
      Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
      Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    return Clusters,Cluster_Centers



  #Normalized - Intra-spatial correlation coefficient Calculation Function
  def nor_intra_sp_corr(input,formed_clusters):
    mylist = []
    intra_sp_corr = []
    Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
    
    for i in range(len(Clusters)):
      mylist = []
      for j in range(len(Clusters['Cluster' + str(i)])):
        corr_coeff = pearson_PM(Clusters['Cluster' + str(i)][j], Cluster_Centers['Cluster_Center' + str(i)])
        #print('i: {}, j: {}, corr_coeff:{}'.format(i, j, corr_coeff))
        mylist.append(corr_coeff)
        average_corr_coeff = sum(mylist) / len(mylist)
      intra_sp_corr.append(average_corr_coeff)
    return intra_sp_corr



      # Non-normalized Spatial Correlation Calculation
  def sp_corr(input,formed_clusters,normalize=False):
    inter_sp_corr = []
    avg_cluster = {}

    if normalize == False:
      #input = handle_missing_values(input)
      Clusters = n_nor_get_clusters(input,formed_clusters)
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_sp_corr(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
            mat[i].iloc[j] = corr_coeff
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
          

    else:
      # trans_data = datatransformation(input)

      # # Data Normalization
      # trans_data = datanormalization(trans_data)

      # # Adding class centers and cluster numbers as columns to the dataframe
      # trans_data['Cluster'] = formed_clusters

      # Rearranging the columns in the dataframe
      # trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
    
      Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

      #Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
      #trans_data = trans_data.reset_index()

    
      mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
      for i in range(len(Clusters)):
        avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
      for i in range(len(Clusters)):
        for j in range(len(Clusters)):
          if i == j:
            a = n_nor_intra_sp_corr(input,formed_clusters)
            mat[i].iloc[j] = round(a[i],2)
          else:
            corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
            mat[i].iloc[j] = round(corr_coeff, 2)
            #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
          

    return mat

  return sp_corr(input,formed_clusters,True)



#################################################################################################
#################################################################################################



###########################  Silhouette Computation  #######################################


def silhouette_score1(input, formed_clusters, *, metric="euclidean", sample_size=None, random_state=None, **kwds):

  trans_data = datatransformation(input)

  trans_data = datanormalization(trans_data)

  trans_data['Cluster'] = formed_clusters

  X1 = data

  if sample_size is not None:
      X1, formed_clusters = check_X_y(X1, formed_clusters, accept_sparse=["csc", "csr"])
      random_state = check_random_state(random_state)
      indices = random_state.permutation(X1.shape[0])[:sample_size]
      if metric == "precomputed":
          X1, formed_clusters = X1[indices].T[indices].T, formed_clusters[indices]
      else:
          X1, formed_clusters = X1[indices], formed_clusters[indices]
  return np.mean(silhouette_samples(X1, formed_clusters, metric=metric, **kwds))

#return silhouette_score1(input, formed_clusters)


In [11]:
#silhouette_score1(data,formed_clusters)

In [12]:
#ST_CORRELATION(data,formed_clusters,True)

In [13]:
ST_RMSE(data,formed_clusters,True)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


,0,1,2,3,4,5,6
0,12.79,25.67,28.91,24.38,13.84,35.76,24.56
1,25.67,12.38,33.03,35.08,26.17,12.49,19.84
2,28.91,33.03,19.06,23.91,20.66,34.5,17.2
3,24.38,35.08,23.91,18.14,27.96,41.81,30.74
4,13.84,26.17,20.66,27.96,13.95,32.75,16.21
5,35.76,12.49,34.5,41.81,32.75,12.99,21.1
6,24.56,19.84,17.2,30.74,16.21,21.1,14.25
